In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import concurrent.futures

# MAKSIMAL PEKERJA SIMULTAN (BISA DISESUAIKAN)
MAX_WORKERS = 10

def get_fakultas_prodi_list():
    # Fungsi ini tidak berubah
    prodi_list = []
    try:
        url_nav = "https://pta.trunojoyo.ac.id/c_search/byfac"
        r = requests.get(url_nav, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        sidebar_nav = soup.select_one('div.box.sidebar_nav')
        if not sidebar_nav: return []
        fakultas_items = sidebar_nav.select_one('ul').find_all('li', recursive=False)
        for item_fakultas in fakultas_items:
            anchor_fakultas = item_fakultas.find('a', recursive=False)
            if not anchor_fakultas: continue
            nama_fakultas = anchor_fakultas.get_text(strip=True)
            ul_prodi = item_fakultas.find('ul')
            if not ul_prodi: continue
            for link_prodi in ul_prodi.select('li a'):
                nama_prodi = link_prodi.get_text(strip=True)
                href = link_prodi.get('href')
                prodi_id = href.strip('/').split('/')[-1]
                if prodi_id.isdigit():
                    prodi_list.append({
                        "id_prodi": int(prodi_id),
                        "nama_prodi": nama_prodi,
                        "nama_fakultas": nama_fakultas
                    })
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar prodi: {e}")
    return prodi_list

def scrape_jurnal_detail(jurnal_url):
    # Fungsi ini tidak berubah
    try:
        response = requests.get(jurnal_url, timeout=15)
        response.raise_for_status()
        isi = BeautifulSoup(response.content, "html.parser").select_one('div#content_journal')
        if not isi: return None
        judul = isi.select_one('a.title').text.strip()
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1].strip()
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1].strip()
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(':')[1].strip()
        abstract_paragraphs = isi.select('p[align="justify"]')
        text_indo_mentah = abstract_paragraphs[0].text if len(abstract_paragraphs) > 0 else ""
        abstrak_indonesia = re.sub(r'\s+', ' ', text_indo_mentah).strip()
        text_inggris_mentah = abstract_paragraphs[1].text if len(abstract_paragraphs) > 1 else ""
        abstrak_inggris = re.sub(r'\s+', ' ', text_inggris_mentah).strip()
        return {
            "penulis": penulis, "judul": judul, "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua, "abstrak": abstrak_indonesia,
            "abstrak_inggris": abstrak_inggris
        }
    except requests.exceptions.RequestException:
        return None

# --- PERUBAHAN UTAMA ADA DI FUNGSI BERIKUT ---
def scrape_prodi(prodi):
    """
    FUNGSI MANAJER: Sekarang akan mengembalikan baris data kosong
    jika tidak ada jurnal yang ditemukan.
    """
    jurnal_urls = []
    page = 1
    while len(jurnal_urls) < 5:
        try:
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi['id_prodi']}/{page}"
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals_on_page = soup.select('li[data-cat="#luxury"] a.gray.button')
            if not jurnals_on_page: break
            for a_tag in jurnals_on_page:
                if len(jurnal_urls) < 5:
                    jurnal_urls.append(a_tag['href'])
                else: break
            page += 1
        except requests.exceptions.RequestException:
            break

    # --- BAGIAN YANG DIMODIFIKASI ---
    # Jika setelah mencari, tidak ada URL jurnal yang ditemukan
    if not jurnal_urls:
        print(f"✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} | Ditemukan 0 jurnal.")
        # Buat satu baris data placeholder
        empty_row = {
            'id_prodi': prodi['id_prodi'],
            'nama_prodi': prodi['nama_prodi'],
            'nama_fakultas': prodi['nama_fakultas'],
            'judul': 'Tidak ada jurnal',
            'penulis': None,
            'pembimbing_pertama': None,
            'pembimbing_kedua': None,
            'abstrak': None,
            'abstrak_inggris': None
        }
        return [empty_row] # Kembalikan sebagai list berisi satu dictionary
    # --- AKHIR MODIFIKASI ---

    # Jika ada URL, proses berjalan seperti biasa
    all_jurnal_data = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        hasil_scrape = executor.map(scrape_jurnal_detail, jurnal_urls)
        for hasil in hasil_scrape:
            if hasil:
                hasil['id_prodi'] = prodi['id_prodi']
                hasil['nama_prodi'] = prodi['nama_prodi']
                hasil['nama_fakultas'] = prodi['nama_fakultas']
                all_jurnal_data.append(hasil)
    
    print(f"✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} | Berhasil mengambil {len(all_jurnal_data)} jurnal.")
    return all_jurnal_data

def main():
    # Fungsi ini tidak berubah
    start_time = time.time()
    daftar_prodi = get_fakultas_prodi_list()
    if not daftar_prodi: return pd.DataFrame()

    final_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results_per_prodi = executor.map(scrape_prodi, daftar_prodi)
        for list_jurnal in results_per_prodi:
            final_results.extend(list_jurnal)

    df = pd.DataFrame(final_results)
    if not df.empty:
        df = df[['nama_fakultas', 'id_prodi', 'nama_prodi', 'judul', 'penulis', 'pembimbing_pertama', 'pembimbing_kedua', 'abstrak', 'abstrak_inggris']]
    
    df.to_csv("pta_final_concurrent_lengkap.csv", index=False)
    
    end_time = time.time()
    print("\n✅ Proses scraping selesai.")
    print(f"Total baris data yang dihasilkan: {len(df)}.")
    print(f"Total waktu eksekusi: {end_time - start_time:.2f} detik.")
    
    return df

# if __name__ == "__main__":
#     df_hasil = main()
#     df_hasil

In [2]:
main()

C:\laragon\bin\python\python-3.10\lib\site-packages\soupsieve\css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


✔️ Selesai: Pertanian - Agribisnis | Berhasil mengambil 5 jurnal.


✔️ Selesai: Pertanian - Manajemen Sumberdaya Perairan | Berhasil mengambil 5 jurnal.


✔️ Selesai: Pertanian - Teknologi Industri Pertanian | Berhasil mengambil 5 jurnal.


✔️ Selesai: Hukum - Ilmu Hukum | Berhasil mengambil 5 jurnal.
✔️ Selesai: Pertanian - Agroteknologi | Berhasil mengambil 5 jurnal.


...

✔️ Selesai: Ekonomi Dan Bisnis - Magister Ilmu Ekonomi | Berhasil mengambil 5 jurnal.


✔️ Selesai: Teknik - Teknik Industri | Berhasil mengambil 5 jurnal.


✔️ Selesai: Teknik - Teknik Informatika | Berhasil mengambil 5 jurnal.


✔️ Selesai: Teknik - Manajemen Informatika | Berhasil mengambil 5 jurnal.


✔️ Selesai: Teknik - Sistem Informasi | Ditemukan 0 jurnal.
✔️ Selesai: Teknik - Teknik Mesin | Ditemukan 0 jurnal.
